# Tiny TextGrad Examples: Basic Prompt Optimization

This notebook demonstrates how to use Tiny TextGrad to optimize a prompt for a given set of inputs. The prompt is optimized using a simple text loss function that focuses on clarity, accuracy, and handling of cases where the answer is not in the text.


In [5]:
import random
from textwrap import dedent

from tinytextgrad.llm import call_llm
from tinytextgrad.prompt import (
    optimize_prompt,
)
from tinytextgrad.ttg import OptimizationResult


def call_llm_with_prompt(
    prompt: OptimizationResult,
    inputs: list[str],
) -> list[str]:
    """
    Calls the LLM with the prompt and inputs.
    Returns a list of strings representing the generated text.
    """
    results = [
        call_llm(
            prompt=prompt.variable.value,
            prompt_input=input,
            model=prompt.model,
            temperature=prompt.temperature,
            max_tokens=prompt.max_tokens,
            top_p=prompt.top_p,
            frequency_penalty=prompt.frequency_penalty,
        )
        for input in inputs
    ]
    for i, result in enumerate(results):
        print(f"Input {i+1}:")
        print(inputs[i])
        print(f"Output {i+1}:")
        print(result)
    return results

In [6]:
initial_prompt = dedent("""
Given some text and a question, determine if the text
contains the answer to the question
""").strip()

data = [
    "Text: The cat is on the mat. Question: Where is the cat?",
    "Text: The sky is blue. Question: What color is the grass?",
    "Text: Paris is the capital of France. Question: What is the capital of Germany?",
    "Text: The Earth orbits the Sun. Question: What does the Earth orbit?",
    "Text: Water freezes at 0 degrees Celsius. Question: At what temperature does water boil?",
    "Text: Beethoven composed many symphonies. Question: Who composed the Fifth Symphony?",
    "Text: Elephants are the largest land animals. Question: What is the largest land animal?",
    "Text: Shakespeare wrote 'Romeo and Juliet.' Question: Who wrote 'Hamlet'?",
    "Text: Humans have 206 bones. Question: How many bones do humans have?",
    "Text: Coffee is typically grown in tropical regions. Question: Where is coffee typically grown?",
]
inputs = random.sample(data, 5)

result = optimize_prompt(
    initial_prompt,
    "gpt-3.5-turbo",
    "gpt-4o",
    inputs,
    num_iterations=1,
)

print("\n\nFinal optimized TEXT_CONTAINS_ANSWER_PROMPT:")
print(result.variable.value)

TEXT_CONTAINS_ANSWER_PROMPT = result

2024-07-12 09:15:34.768 | DEBUG    | tinytextgrad.ttg:optimize_text:174 - ∇ 1. Current prompt:

Given some text and a question, determine if the text
contains the answer to the question
2024-07-12 09:15:37.518 | DEBUG    | tinytextgrad.ttg:forward:131 - ∇ Evaluation input:

Text:
Given some text and a question, determine if the text
contains the answer to the question

Results:
Input: Text: The sky is blue. Question: What color is the grass?
Output: The text does not contain the answer to the question.
Input: Text: Coffee is typically grown in tropical regions. Question: Where is coffee typically grown?
Output: Yes, the text contains the answer to the question.
Input: Text: Beethoven composed many symphonies. Question: Who composed the Fifth Symphony?
Output: Yes
Input: Text: Shakespeare wrote 'Romeo and Juliet.' Question: Who wrote 'Hamlet'?
Output: No, the text does not contain the answer to the question.
Input: Text: Water freezes at 0 degrees Celsius. Question: At what temperature 



Final optimized TEXT_CONTAINS_ANSWER_PROMPT:
Given a passage of text and a question, determine if the text contains a complete and direct answer to the question. If the text does contain the answer, respond with "Yes, the text contains the answer to the question" and provide a brief explanation. If the text does not contain the answer, respond with "No, the text does not contain the answer to the question" and suggest where the information might be found. If the text provides a partial or closely related answer, indicate this clearly in your response.


In [8]:
inputs = random.sample(data, 5)
call_llm_with_prompt(
    prompt=TEXT_CONTAINS_ANSWER_PROMPT,
    inputs=inputs,
)

Input 1:
Text: Elephants are the largest land animals. Question: What is the largest land animal?
Output 1:
Yes, the text contains the answer to the question. The text states, "Elephants are the largest land animals," directly answering that elephants are the largest land animals.
Input 2:
Text: Shakespeare wrote 'Romeo and Juliet.' Question: Who wrote 'Hamlet'?
Output 2:
No, the text does not contain the answer to the question. The information about who wrote "Hamlet" can be found in a biography of William Shakespeare or in a literary analysis of his works.
Input 3:
Text: Coffee is typically grown in tropical regions. Question: Where is coffee typically grown?
Output 3:
Yes, the text contains the answer to the question. The text states, "Coffee is typically grown in tropical regions," directly answering where coffee is typically grown.
Input 4:
Text: Humans have 206 bones. Question: How many bones do humans have?
Output 4:
Yes, the text contains the answer to the question. The text di

['Yes, the text contains the answer to the question. The text states, "Elephants are the largest land animals," directly answering that elephants are the largest land animals.',
 'No, the text does not contain the answer to the question. The information about who wrote "Hamlet" can be found in a biography of William Shakespeare or in a literary analysis of his works.',
 'Yes, the text contains the answer to the question. The text states, "Coffee is typically grown in tropical regions," directly answering where coffee is typically grown.',
 'Yes, the text contains the answer to the question. The text directly states, "Humans have 206 bones," providing the exact number of bones in humans.',
 'Yes, the text contains the answer to the question. The answer is "the Sun."']

In [9]:
initial_prompt = """
Generate a set of relevant questions and their
corresponding answers about the given text. Ensure the
questions cover a mix of factual, analytical, and
application-based types to provide both surface-level
and in-depth knowledge of the subject.
"""

data = [
    "The process of cellular respiration converts biochemical energy from nutrients into adenosine triphosphate (ATP), and releases waste products. Cellular respiration is a set of metabolic reactions and processes that take place in the cells of organisms to convert chemical energy from oxygen molecules or nutrients into ATP, and then release waste products.",
    "The first human heart transplant was performed by Dr. Christiaan Barnard on December 3, 1967, in Cape Town, South Africa. The patient, Louis Washkansky, lived for 18 days after the surgery.",
    "Rosalind Franklin was a British biophysicist and X-ray crystallographer whose work was critical in the understanding of the molecular structures of DNA, RNA, and viruses. She is best known for her photograph of DNA, known as Photo 51, which contributed significantly to the discovery of the DNA double helix by Watson and Crick.",
    "The Great Wall of China is a series of fortifications made of various materials, including stone, brick, tamped earth, wood, and other materials. It was built along the northern borders of China to protect against invasions and raids from various nomadic groups. The most well-known sections were built by the Ming Dynasty (1368-1644).",
    "The Theory of General Relativity, formulated by Albert Einstein, describes the gravitational force as a curvature of spacetime caused by mass and energy. One of its most famous predictions is the bending of light around massive objects, which has been confirmed through various experiments, including the observation of a solar eclipse in 1919.",
    "Marie Curie was a Polish-born physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize and the only person to win Nobel Prizes in two different scientific fields—Physics (1903) and Chemistry (1911). Her discoveries included the elements polonium and radium.",
    "The internet is a global network of interconnected computers that communicate via standardized protocols. It enables a wide range of services, including the World Wide Web, email, and file sharing. The internet has revolutionized communication, commerce, and access to information.",
]
inputs = data[:1]

result = optimize_prompt(
    initial_prompt,
    "gpt-3.5-turbo",
    "gpt-4o",
    inputs,
    num_iterations=1,
)

print("\n\nFinal optimized GENERATE_QA_PAIRS_PROMPT:\n")
print(result.variable.value)

GENERATE_QA_PAIRS_PROMPT = result

2024-07-12 09:16:57.534 | DEBUG    | tinytextgrad.ttg:optimize_text:174 - ∇ 1. Current prompt:


Generate a set of relevant questions and their
corresponding answers about the given text. Ensure the
questions cover a mix of factual, analytical, and
application-based types to provide both surface-level
and in-depth knowledge of the subject.

2024-07-12 09:17:03.476 | DEBUG    | tinytextgrad.ttg:forward:131 - ∇ Evaluation input:

Text:

Generate a set of relevant questions and their
corresponding answers about the given text. Ensure the
questions cover a mix of factual, analytical, and
application-based types to provide both surface-level
and in-depth knowledge of the subject.


Results:
Input: The process of cellular respiration converts biochemical energy from nutrients into adenosine triphosphate (ATP), and releases waste products. Cellular respiration is a set of metabolic reactions and processes that take place in the cells of organisms to convert chemical energy from oxygen molecul



Final optimized GENERATE_QA_PAIRS_PROMPT:

Generate a balanced set of relevant questions and their corresponding answers about the given text. Ensure the questions cover a mix of factual, analytical, and application-based types to provide both surface-level and in-depth knowledge of the subject. If any required information is not available in the text, clearly state that it is not provided.


In [10]:
inputs = data[2:3]
call_llm_with_prompt(
    prompt=GENERATE_QA_PAIRS_PROMPT,
    inputs=inputs,
)

Input 1:
Rosalind Franklin was a British biophysicist and X-ray crystallographer whose work was critical in the understanding of the molecular structures of DNA, RNA, and viruses. She is best known for her photograph of DNA, known as Photo 51, which contributed significantly to the discovery of the DNA double helix by Watson and Crick.
Output 1:
1. What was Rosalind Franklin's primary area of expertise?
Answer: Rosalind Franklin was a British biophysicist and X-ray crystallographer.

2. What is Rosalind Franklin best known for in the field of molecular biology?
Answer: Rosalind Franklin is best known for her photograph of DNA, known as Photo 51, which contributed significantly to the discovery of the DNA double helix by Watson and Crick.

3. How did Rosalind Franklin's work impact the understanding of DNA, RNA, and viruses?
Answer: Rosalind Franklin's work was critical in the understanding of the molecular structures of DNA, RNA, and viruses through X-ray crystallography.

4. What spec

["1. What was Rosalind Franklin's primary area of expertise?\nAnswer: Rosalind Franklin was a British biophysicist and X-ray crystallographer.\n\n2. What is Rosalind Franklin best known for in the field of molecular biology?\nAnswer: Rosalind Franklin is best known for her photograph of DNA, known as Photo 51, which contributed significantly to the discovery of the DNA double helix by Watson and Crick.\n\n3. How did Rosalind Franklin's work impact the understanding of DNA, RNA, and viruses?\nAnswer: Rosalind Franklin's work was critical in the understanding of the molecular structures of DNA, RNA, and viruses through X-ray crystallography.\n\n4. What specific contribution did Rosalind Franklin's Photo 51 make to the field of genetics?\nAnswer: Rosalind Franklin's Photo 51 provided crucial evidence that helped Watson and Crick determine the double helical structure of DNA.\n\n5. Apart from DNA, what other areas did Rosalind Franklin's research focus on?\nAnswer: Rosalind Franklin's rese